In [9]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from user_agent import generate_user_agent
import numpy as np
import pandas as pd

In [14]:
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
url = 'http://notelections.online'
start_link = 'http://notelections.online/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'

In [36]:
start_page = requests.get(start_link, headers=headers)
soup = BeautifulSoup(start_page.text, 'html.parser')

# Получение списка полей

In [53]:
fields = ['Регион', 'ТИК', 'УИК']
fields_list = soup.find('table', class_='table-fixed-columns').find('tbody').findAll('tr')
for field in fields_list:
    fields.append(field.findAll('td')[1].text)
fields

['Регион',
 'ТИК',
 'УИК',
 'Число избирателей, включенных в список избирателей ',
 'Число избирательных бюллетеней, полученных участковой избирательной комиссией',
 'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно',
 'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования',
 'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования',
 'Число погашенных избирательных бюллетеней',
 'Число избирательных бюллетеней в переносных ящиках для голосования',
 'Число бюллетеней в стационарных ящиках для голосования',
 'Число недействительных избирательных бюллетеней',
 'Число действительных избирательных бюллетеней',
 'Число утраченных избирательных бюллетеней',
 'Число избирательных бюллетеней, не учтенных при получении ',
 'Бабурин Сергей Николаевич',
 'Грудинин Павел Николаевич',
 'Жириновский Владимир Вольфович',
 'Путин Владимир Владимирович',
 'Собчак Ксения Анатольевна',
 'Сурайкин Максим Алек

# Парсинг данных

In [225]:
def sub_region_table(subregion_page, subregion_name, region_name):
    sub_region_soup = BeautifulSoup(subregion_page.text, 'html.parser')
    table = sub_region_soup.find('table', class_='table-fixed-columns')
    head = table.findAll('th')[3:]
    rows = table.find('tbody').findAll('tr')
    data = [[region_name, subregion_name, i.text] for i in head]
    for row in rows:
        td = row.findAll('td')[3:]           
        for ind, t in enumerate(td):
            text = t.find('nobr').text
            data[ind].append(text)
            
    return data

In [257]:
def region_table(region_page, region_name):
    region_soup = BeautifulSoup(region_page.text, 'html.parser')
    subregion_li = region_soup.find('table', class_='table-fixed-columns').findAll('th')[3:]
    data = []
    for subregion in subregion_li:
        subregion_name = subregion.find('a').text
        link_subregion = subregion.find('a').attrs['href']
        subregion_page = requests.get(URL + link_subregion, headers=headers)
        data.extend(sub_region_table(subregion_page, subregion_name, region_name))  
    return data

In [279]:
%%time
region_li = soup.find('table', class_='table-fixed-columns').findAll('th')[3:]
data = []
for region in tqdm(region_li[:-2]):
    link = region.find('a').attrs['href']
    region_name = region.find('a').text
    region_page = requests.get(url + link, headers=headers)
    data.extend(region_table(region_page, region_name))
    
for region in tqdm(region_li[-2:]):
    link = region.find('a').attrs['href']
    region_name = region.find('a').text
    region_page = requests.get(url + link, headers=headers)
    data.extend(sub_region_table(region_page, "", region_name))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.89s/it]

Wall time: 1h 25min 31s


In [291]:
df = pd.DataFrame(data=data, columns=fields)

In [295]:
df.to_csv('parser_data.csv', index=False)

In [297]:
df.head()

,Регион,ТИК,УИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Республика Адыгея (Адыгея),Адыгейская,УИК №1,2256,2181,0,2107,62,12,62,...,0,0,0,137,32,1977,14,0,1,5
1,Республика Адыгея (Адыгея),Адыгейская,УИК №2,2700,2633,0,2575,41,17,41,...,0,0,15,86,65,2389,13,5,6,15
2,Республика Адыгея (Адыгея),Адыгейская,УИК №3,2858,2752,0,2664,75,13,75,...,0,0,1,62,13,2645,6,3,4,0
3,Республика Адыгея (Адыгея),Адыгейская,УИК №4,2066,2034,0,1857,142,35,142,...,0,0,5,288,12,1642,21,6,2,2
4,Республика Адыгея (Адыгея),Адыгейская,УИК №5,700,714,0,676,11,27,11,...,0,0,2,44,6,624,6,0,0,2
